# Transfer Service

[Storage Transfer Service](https://cloud.google.com/storage-transfer-service)^[I SSB kaller vi tjenesten for *Transfer Service*, men du kan oppleve at Google kaller den litt forskjellige ting. Den blir omtalt som *Storage Transfer Service* noen steder, mens i *Google Cloud Console* blir den omtalt som *Data Transfer* eller *Storage Transfer*] er en Google-tjeneste for å flytte data mellom lagringsområder. I SSB bruker vi hovedsakelig tjenesten til å: 

1. Flytte data mellom et Dapla-team sine bøtter på Dapla og deres lagringsområde i prodsonen. 

2. Flytte data mellom bøtter på Dapla^[Flytting av data mellom bøtter krever at prosjektets Transfer Service har tilgang til begge bøttene.].

Tjenesten støtter både automatiserte og ad-hoc overføringer, og den inkluderer et brukergrensesnitt for å sette opp og administrere overføringene i [Google Cloud Console (GCC)](https://console.cloud.google.com/).


## Tilgangsstyring

Tilgangsstyringen til data gjelder også for overføringer av data med Transfer Service. Det betyr at du må ha tilgang til dataene du skal sette opp overføringsjobber for. Siden tilgang til kildedata kun er forbeholdt teamets **data-admins** betyr dette at man må gi seg selv JIT-tilgang til kildedata for å kunne sette opp overføringer. Annen data enn kildedata er tilgjengelig for alle i tilgangsgruppen **developers**, og det er ikke nødvendig å gi seg selv JIT-tilgang til disse dataene for å benytte Transfer Service.

Ved bruk av Transfer Service for overføring av data mellom bakke og sky så er det satt opp en dedikert mappe for dette i prodsonen. Også her har **developers** og **data-admins** tilgang til ulike områder. **developers** har tilgang til en mappe som heter `standard`, mens **data-admins** har tilgang til en mappe som heter `kilde`.

::: {.callout-note}
# Personlig bruker vs systembruker
Når du setter opp en overføringsjobb med Transfer Service så setter du opp en jobb som kjøres av en systembruker^[Systembrukere heter *Service Accounts* på engelsk og blir ofte referert til som *SA*-er i dagligtale.] og ikke din egen personlige bruker. Dette er spesielt viktig å være klar over når man setter opp automatiserte overføringsjobber. Dette betyr blant annet at automatiske jobber, som settes opp mens en tilgang til kildedata er aktiv, vil fortsette og gå selv om din brukers tilgang ikke lenger er aktiv. 
:::

## Bruk av Transfer Service

Grensesnittet for å sette opp overføringsjobber i Transfer Service er tilgjengelig i *Google Cloud Console* (GCC).

:::: {.columns}

::: {.column width="50%"}
1. Gå inn på [Google Cloud Console](https://console.cloud.google.com) i en nettleser.
2. Sjekk øverst i høyre hjørne at du er logget inn med din SSB-konto (xxx@ssb.no).
3. Velg prosjektet^[Du finner prosjektvelgeren øverst på siden, til høyre for teksten **Google Cloud**. Deretter kan du søke etter team-navnet du skal opprette en jobb for. Til slutt velger du enten kilde- eller standardprosjektet.] som overføringen skal settes opp under. For å velge prosjekt kan du trykke på navnet til prosjektet du vil velge. Da ser du navnet på prosjektet øverst i bildet etterpå. 
4. Etter at du har valgt prosjekt kan du søke etter **Storage Transfer** i søkefeltet øverst på siden, og gå inn på siden.
:::


::: {.column width="50%"}
::: {.callout-note}
# Hva er mitt prosjektnavn? 
Når det opprettes et Dapla-team, så opprettes det flere Google-prosjekter for teamet. Når du skal velge hvilket prosjekt du skal jobbe på i GCC, så følger de en fast navnestruktur. For eksempel så vil et team med navnet **dapla-example** få et standardprosjekt som heter `dapla-example-p`. Det blir også opprettet et kildeprosjekt som heter `dapla-example-kilde-p`. 

:::
:::

::::

Første gang man kommer inn på siden til **Storage Transfer** vil man se en blå knapp med teksten **Set Up Connection**. Når du trykker på denne vil det dukke opp et nytt felt hvor du får valget **Create Pub-Sub Resources**. Dette er noe som bare trengs å gjøre én gang. Trykk på den blå **Create**-knappen, og deretter trykk på **Close** lenger nede.

5. I navigasjonsmenyen til venstre trykk `Transfer jobs`, og deretter trykk på `+ Create transfer job` øverst på siden for å opprette en ny overføringsjobb. Da får du opp bildet som vist i @fig-transfer-tilsky.

![Opprett overføringsjobb i Google Cloud Console.](../images/data-transfer-tilsky.png){fig-alt="Bilde som viser Create a transfer job i Google Cloud Console." #fig-transfer-tilsky}

Videre vil det variere om man skal overføre data mellom bøtter eller mellom Dapla og prodsonen. Under forklarer vi begge fremgangsmåtene.



In [1]:
# | echo: false

# Lager et diagram
from diagrams import Cluster, Diagram, Edge
from diagrams.gcp.storage import GCS
from diagrams.generic.storage import Storage
from diagrams.generic.blank import Blank

graph_attr_cluster = {
    "fontsize": "8",
    "bgcolor": "lightyellow",
    "gradientangle": "360"
    # "style": 'filled',
    # "label": "test",
    # "fontcolor": "blue",
}

graph_attr_diagram = {
    # "fontsize": "45",
    "bgcolor": "transparent"
}

with Diagram(
    "", show=False, filename="ts-kilde", graph_attr={}, direction="TB"
):
    with Cluster("DAPLA", graph_attr={"fontsize": "20"}):
        Blank("")

        with Cluster("kilde", graph_attr=graph_attr_cluster):
            dkf = GCS("synk_ned")
            dkt = GCS("synk_opp")

        with Cluster("standard", graph_attr=graph_attr_cluster):
            dpf = GCS("synk_ned")
            dpt = GCS("synk_opp")

    with Cluster("PRODSONE", graph_attr={"fontsize": "20", "bgcolor": "lightgrey"}):
        Blank("/ssb/cloud_sync/")

        with Cluster("kilde", graph_attr=graph_attr_cluster):
            okf = Storage("frasky")
            okt = Storage("tilsky")

        with Cluster("standard", graph_attr=graph_attr_cluster):
            opf = Storage("frasky")
            opt = Storage("tilsky")

        edge_admins = Edge(xlabel="data-admins")
        edge_devs = Edge(xlabel="developers")

    dkf >> edge_admins >> okf
    okt >> dkt
    dpf >> edge_devs >> opf
    opt >> dpt


In [3]:
#| echo: false

# Cropper bildet så det skal ta mindre plass
from PIL import Image

# Open the image
img = Image.open("ts-kilde.png")

# Define the coordinates for the cropping box
# The box is defined as (left, upper, right, lower)
left = 200
upper = 200
right = 1150
lower = 800
cropping_box = (left, upper, right, lower)

# Crop the image
cropped_img = img.crop(cropping_box)

# Save the cropped image
cropped_img.save("ts-kilde-cropped.png")

### Prodsonen og Dapla

Overføring mellom bakke og sky er en overføring av data mellom en bøtte på Dapla og en mappe i prodsonen. Siden tilgangsstyring til kildedata er strengere enn tilgangsstyring til annen data, så er det det to litt fremgangsmåter for å sette opp overføringsjobber for disse. 

Siden stegene er litt forskjellig avhengig av om man skal flytte kildedata eller annen data, så deler vi denne delen i to. @fig-ts-bakke-sky viser hvordan dette er satt opp. Kildeprosjektet på Dapla har en **synk-opp**-bøtte for å flytting av data fra prodsonen til Dapla, og den har en **synk-ned**-bøtte for å flytte data fra Dapla til prodsonen. Standardprosjektet på Dapla har også en **synk-opp**-bøtte for å flytte data fra prodsonen til Dapla, og den har en **synk-ned**-bøtte for å flytte data fra Dapla til prodsonen.

![Overføring av data mellom prodsonen og Dapla.](ts-kilde-cropped.png){fig-alt="Diagram av mapper i prodsonen og bøtter på Dapla, og hvordan overføringene kan skje mellom de." #fig-ts-bakke-sky}



6. I fanen **Get started** velger du:
    - **Fra bakke til sky**:  
    Under *Source type* velges **POSIX filesystem** og  under *Destination type* velges **Google Cloud Storage**.
    - **Fra sky til bakke**:  
    Under *Source type* velges **Google Cloud Storage** og  under *Destination type* velges **POSIX filesystem**.

7. Nedtrekkslisten **Agent pool** skal normalt bare ha ett valg: **transfer_service_default**. Velg denne.
8. I feltet **Source directory path** skal man skrive inn `frasky` hvis overføringen er fra sky til bakke, og `tilsky` hvis overføringen er fra bakke til sky. Trykk `Next step`
9. Velg en destinasjon for overføringsjobben. Trykk på `Browse` og velg bøtten med navn som passer til `ssb-prod-<teamnavn>-data-synk-opp`. Vi anbefaler at du også oppretter en mappe inne i denne bøtten. Det gjøres ved å trykke på mappeikonet med et `+`-tegn foran. Skriv inn et passende mappenavn og trykk `Select` i bunnen av siden. Trykk deretter `Next step`
10. Neste steg "Choose how and when to run this job" er opp til brukeren å bestemme. Hvis man f.eks. velger at Data Transfer skal overføre data en gang i uken, vil den kun starte en overføring hvis det finnes nye data. Trykk `Next step`
11. Beskriv overføringsjobben, f.eks: "Flytter data for &lt;teamnavn&gt; til sky". Resten av feltene er opp til brukeren å bestemme. Standardverdiene er OK.

Trykk til slutt på den blå `Create`-knappen. Du vil kunne se kjørende jobber under menyen `Transfer jobs`.

For å sjekke om data har blitt overført, skriv inn `cloud storage` i søkefeltet øverst på siden og trykk på det første valget som kommer opp. Her vil du finne en oversikt over alle teamets bøtter, deriblant en med navn `ssb-prod-<teamnavn>-data-synk-opp`. Når overføringsjobben er ferdig vil du kunne finne igjen dataene i den mappen som ble definert i stegene overnfor.




Transfer Service er organisert i **kilder** og **mål**. En **kilde** er et lagringsområde som data skal flyttes fra, mens et **mål** er et lagringsområde som data skal flyttes til. I SSB bruker vi Transfer Service til å flytte data mellom bøtter på Dapla og lagringsområder i prodsonen.

## Prodsonen

Overføring av data mellom prodsonen og Dapla skjer via dedikerte lagringsområder i prodsonen. Når et team opprettes 

```{.yaml filename="/ssb/cloud_sync/dapla-example-p/"}
dapla-example-p
│
├── kilde
│   │
│   │── tilsky
│   │
│   └── frasky
│
└── standard
    │
    │── tilsky
    │
    └── frasky
```